In [ ]:
CREATE TABLE stock(
    id SERIAL PRIMARY KEY,
    symbol TEXT NOT NULL, 
    name TEXT NOT NULL, 
    exchange TEXT NOT NULL,
    is_etf BOOLEAN NOT NULL
);

CREATE TABLE mention (
    stock_id INTEGER,
    dt TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    message TEXT NOT NULL,
    source TEXT NOT NULL, 
    url TEXT NOT NULL,
    PRIMARY KEY (stock_id, dt),
    CONSTRAINT fk_mention_stock FOREIGN KEY (stock_id) REFERENCES stock (id)
);

CREATE INDEX ON mention (stock_id, dt DESC);
SELECT create_hypertable('mention', 'dt');

CREATE TABLE etf_holding (
    etf_id INTEGER NOT NULL,
    holding_id INTEGER NOT NULL,
    dt DATE NOT NULL,
    shares NUMERIC,
    weight NUMERIC,
    PRIMARY KEY (etf_id, holding_id, dt),
    CONSTRAINT fk_etf FOREIGN KEY (etf_id) REFERENCES STOCK (id),
    CONSTRAINT fk_holding FOREIGN KEY (holding_id) REFERENCES stock (id)
);

CREATE TABLE stock_price (
    stock_id INTEGER NOT NULL,
    dt TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    open NUMERIC (6,2) NOT NULL,
    high NUMERIC (6,2) NOT NULL,
    low NUMERIC (6,2) NOT NULL,
    close NUMERIC (6,2) NOT NULL,
    volume NUMERIC NOT NULL,
    PRIMARY KEY (stock_id, dt),
    CONSTRAINT fk_stock FOREIGN KEY (stock_id) REFERENCES stock (id)
);

CREATE INDEX ON stock_price (stock_id, dt DESC);

SELECT create_hypertable('stock_price', 'dt');

In [5]:
import streamlit as st
import pandas as pd
import numpy as np
import requests
import tweepy
import config 
import psycopg2, psycopg2.extras
import plotly.graph_objects as go

In [6]:
connection = psycopg2.connect(host='localhost', database='etfdb', user='postgres', password='secret', port='4747')

cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [10]:
data = pd.read_sql("""
     select date(day) as day, open, high, low, close
     from daily_bars
     where stock_id = (select id from stock where symbol = 'MSFT')
     order by day asc""", connection, params={'stock':'id'}

SyntaxError: unexpected EOF while parsing (<ipython-input-10-e270e46b2495>, line 5)

In [ ]:
data = pd.read_sql("""
        select date(day) as day, open, high, low, close
        from daily_bars
        where stock_id = (select id from stock where UPPER(symbol) = %s) 
        order by day asc""", connection, params=(symbol.upper(),))